Hello, Let's connect on Linkedin.
https://www.linkedin.com/in/viviankaun/

In [ ]:
!pip install openai


In [ ]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)

In [ ]:
!pip install llama-index --upgrade

In [ ]:
!pip install pypdf

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_key

###When we only read one document
documents = SimpleDirectoryReader("./").load_data("CaliforniaDMV.pdf")
It acctully read all of pdfs in the path, so delete all of pdf.
and upload the file we need.

In [ ]:

%rm -rf *.pdf
%ls -l


total 8
drwxr-xr-x 2 root root 4096 Aug  8 08:10 chroma/
drwxr-xr-x 1 root root 4096 Aug  6 13:32 sample_data/


In [ ]:
# Upload files
from google.colab import files
uploaded = files.upload()

# List uploaded files
import os

for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes.')

# Accessing the uploaded file
uploaded_filename = list(uploaded.keys())[0]
print(f'Uploaded file path: /content/{uploaded_filename}')

Saving CaliforniaDMV.pdf to CaliforniaDMV.pdf
User uploaded file "CaliforniaDMV.pdf" with length 54813 bytes.
Uploaded file path: /content/CaliforniaDMV.pdf


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
# Define the service context with a custom chunk size, OpenAI key needed here
service_context = ServiceContext.from_defaults(chunk_size=1000, chunk_overlap=100)
documents = SimpleDirectoryReader("./").load_data("CaliforniaDMV.pdf")



<ipython-input-49-2517d2745411>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(chunk_size=1000, chunk_overlap=100)
Loading files:   0%|          | 0/1 [00:00<?, ?file/s]/usr/local/lib/python3.10/dist-packages/fsspec/implementations/local.py:428: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  return self.f.seek(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/fsspec/implementations/local.py:428: RuntimeWarning: coroutine 'Dispatcher.span.<locals>.async_wrapper' was never awaited
  return self.f.seek(*args, **kwargs)
Loading files: 100%|██████████| 1/1 [00:00<00:00,  4.86file/s]


In [ ]:
!pip install llama-index-vector-stores-chroma

#Renaker：  Got a better answer.

In [ ]:
!pip install torch sentence-transformers

In [ ]:

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)
query_engine = index.as_query_engine()
response = query_engine.query("If I complete both driver education and driver training, Anything I should do before scheduling behind-the-wheel drive test?")
print(response)

Before scheduling your behind-the-wheel drive test, ensure you have completed both driver education and driver training.


In [ ]:

from llama_index.core.postprocessor import SentenceTransformerRerank

# Initialize the Sentence Transformer Reranker
rerank = SentenceTransformerRerank(
    top_n=2,
    model="BAAI/bge-reranker-base"  # Specify the model to use for reranking
)

In [ ]:
query_engine_reranker = index.as_query_engine(
    similarity_top_k = 6,
    node_postprocessors = [rerank],
)
response = query_engine_reranker.query("If I complete both driver education and driver training, Anything I should do before scheduling behind-the-wheel drive test?")
print(response)

Before scheduling your behind-the-wheel drive test, you should ensure that you have practiced driving for at least 50 hours with a California-licensed driver who is at least 25 years old. Out of the 50 hours, ten hours must be completed at night.


#RouterQueryEngine:
## summary_tool, vector_tool, keyword_tool,

In [ ]:
# Import necessary classes from the llama_index package
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, SummaryIndex
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

nodes = Settings.node_parser.get_nodes_from_documents(documents)

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool
from llama_index.core import SimpleKeywordTableIndex

import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("llm7")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context using keywords from Paul"

    ),
)

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions related to the data source",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context related to the data source",
)

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
        keyword_tool,
    ],
)


In [ ]:
response = query_engine.query("Summarize the document in 4 bullet points")
print(response)

- To obtain an instruction permit or driver's license in California, applicants must provide proof of identity, residency, legal name, and social security number, along with completing necessary application steps and tests.
- Minors under 18 must complete a driver education program, have parental approval, and practice driving with a licensed adult before taking the behind-the-wheel test.
- Provisional licenses for drivers under 18 come with restrictions, such as limitations on driving hours and passenger age, with exceptions for medical, school, work, or family-related needs.
- The testing process includes a vision test, knowledge test, and behind-the-wheel test, with specific requirements for the vehicle used during the drive test and regulations regarding the use of technology and recording devices.


In [ ]:
response = query_engine.query("How can I get a driver license ?")
print(response)

To obtain a driver's license, you need to follow these steps:

1. **Eligibility**: If you are under 18, you must be at least 16 years old and have held an instruction permit for at least 6 months or be 18 years old.

2. **Driver Education and Training**: Complete both a driver education program and behind-the-wheel driver training.

3. **Practice Driving**: Accumulate at least 50 hours of practice driving with a California-licensed driver who is at least 25 years old, including 10 hours at night.

4. **Application**: Apply for your driver's license and schedule a behind-the-wheel drive test.

5. **Testing**: Pass the following tests:
   - **Vision Test**: Ensure you can see well enough to drive safely.
   - **Knowledge Test**: Pass a multiple-choice knowledge test. You have three attempts to pass, with a seven-day waiting period for minors after a failure.
   - **Behind-the-Wheel Drive Test**: Demonstrate your ability to safely operate a vehicle.

Make sure to check the DMV website for

In [ ]:
response = query_engine.query(" Could you find the keyword Vision Test ?")
print(response)

The keyword "Vision Test" is mentioned in the overview of the driver’s license tests, specifically as the first test that all applicants must undergo to ensure they can see well enough to drive safely.


In [ ]:
# RouterQueryEngine:
summary_tool, vector_tool, keyword_tool,

In [ ]:
!pip install llama-index-readers-youtube-transcript
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader
# find a youtube video on my home page
youtube_video_file = YoutubeTranscriptReader()
documents = youtube_video_file.load_data(
    ytlinks=["https://www.youtube.com/watch?v=ejeGJHeKn-o"]
)

In [ ]:
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

nodes = Settings.node_parser.get_nodes_from_documents(documents)

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()
chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("llm8")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context using keywords from Paul"

    ),
)

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions related to the data source",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context related to the data source",
)

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
        keyword_tool,
    ],
)


In [ ]:
response = query_engine.query("Summarize the document as a tutorial. ")
print(response)

This tutorial focuses on identifying consecutive streaks in player performance data from the game Halo 5, specifically analyzing win streaks. It begins by explaining the importance of finding streaks for business insights, such as detecting potential cheating.

The tutorial outlines a step-by-step approach using SQL, emphasizing the elegance of solving the problem directly within a database rather than transferring data between systems. Key steps include:

1. **Setting Up the Environment**: Ensure a local PostgreSQL instance is installed and ready for use.

2. **Data Preparation**: The tutorial starts with a dataset containing player match details, including player identifiers, win/loss status, kills, deaths, and match timestamps. It involves joining multiple tables to gather necessary columns.

3. **Using Window Functions**: The tutorial demonstrates how to use window functions to analyze the data. It partitions the data by player and playlist, ordering by completion date to track con

In [ ]:
response = query_engine.query("Summarize the document in 4 bullet points")
print(response)

- The discussion focuses on identifying consecutive streaks in data, such as win streaks or user activity streaks, using both Python and SQL, with an emphasis on the efficiency of SQL queries.
- A dataset from the game Halo 5 is utilized, containing player performance metrics like wins, kills, deaths, and match timestamps to analyze streaks and detect potential outliers or cheating.
- The process involves creating common table expressions (CTEs) to track win/loss changes, using window functions to partition data by player and playlist, and identifying streaks based on game outcomes.
- The final analysis ranks the streaks to highlight significant winning patterns, with the results available for further exploration in a GitHub repository linked in the video.


In [ ]:
response = query_engine.query("How do i rank the streaks to find the longest winning streaks ?")
print(response)

To rank the streaks and find the longest winning streaks, you can use SQL window functions. First, you need to identify the streaks by comparing the current match result with the previous match result for each player. Once you have identified the streaks, you can count the number of consecutive wins for each streak.

Here’s a step-by-step approach:

1. **Identify Streak Changes**: Create a Common Table Expression (CTE) that compares the current match result with the previous match result using the `LAG` function. This will help you determine when a streak starts or ends.

2. **Assign Streak Identifiers**: Use a `CASE` statement to create a column that indicates whether a streak has changed. If the current match result is different from the previous one, it indicates the start of a new streak.

3. **Count Streak Lengths**: Create another CTE that counts the number of records (matches) for each streak using the `ROW_NUMBER` function, partitioned by the player and the streak identifier.



In [ ]:
response = query_engine.query("Can you find keyword CTEs?")
print(response)

Yes, the keyword "CTE" appears in the context as "starter cte" when discussing the use of window functions and partitioning in SQL queries.
